## From "Python Data Science Handbook"

In [1]:
# importing necesary libraries

 

%matplotlib inline

import pandas as pd

import numpy as np

import glob

import os

import datetime

import statsmodels

from matplotlib import pyplot as plt

import seaborn as sns

import sys

import plotly.express as px

import warnings

import plotly

from plotly.subplots import make_subplots

import plotly.graph_objects as go

warnings.filterwarnings("ignore")

plotly.offline.init_notebook_mode(connected=True)

 

from IPython.display import display

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

 

import sys

print(sys.version)

print(sys.path)


3.7.6 (default, Dec 30 2019, 13:29:21) 
[Clang 11.0.0 (clang-1100.0.33.16)]
['/Users/tswiton/dev/data_upskill/e2', '/Users/tswiton/dev/data_upskill', '/usr/local/Cellar/python/3.7.6/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/usr/local/Cellar/python/3.7.6/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/usr/local/Cellar/python/3.7.6/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Users/tswiton/.virtualenvs/data_upskill/lib/python3.7/site-packages', '/Users/tswiton/.virtualenvs/data_upskill/lib/python3.7/site-packages/setuptools-40.8.0-py3.7.egg', '/Users/tswiton/.virtualenvs/data_upskill/lib/python3.7/site-packages/pip-19.0.3-py3.7.egg', '/Users/tswiton/.virtualenvs/data_upskill/lib/python3.7/site-packages/IPython/extensions', '/Users/tswiton/.ipython']


In [2]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=['subject', 'type'])
# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37
# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      33.0  36.8  46.0  36.7  43.0  36.1
     2      26.0  36.5  22.0  37.8  34.0  37.2
2014 1      54.0  37.3  31.0  35.2  23.0  36.4
     2      23.0  37.3  40.0  37.8  50.0  37.6

In [3]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      46.0  36.7
     2      22.0  37.8
2014 1      31.0  35.2
     2      40.0  37.8

In [4]:
pop = {('California', 2000): 33871648,
                    ('California', 2010): 37253956,
                    ('Texas', 2000): 20851820,
                    ('Texas', 2010): 25145561,
                    ('New York', 2000): 18976457,
                    ('New York', 2010): 19378102}
pop = pd.Series(pop)
pop

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

In [5]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

In [6]:
pop['California', 2000]
pop = pop.sort_index()
print(pop)
print(pop['California':'New York'])
flat_pop = pop.reset_index(name="population")
print(flat_pop)
unflat_pop = flat_pop.set_index(['state', 'year'])
print(unflat_pop)

33871648

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64
state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64
        state  year  population
0  California  2000    33871648
1  California  2010    37253956
2    New York  2000    18976457
3    New York  2010    19378102
4       Texas  2000    20851820
5       Texas  2010    25145561
                 population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561


In [7]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      33.0  36.8  46.0  36.7  43.0  36.1
     2      26.0  36.5  22.0  37.8  34.0  37.2
2014 1      54.0  37.3  31.0  35.2  23.0  36.4
     2      23.0  37.3  40.0  37.8  50.0  37.6

In [8]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     29.5  36.65  34.0  37.25  38.5  36.65
2014     38.5  37.30  35.5  36.50  36.5  37.00

In [9]:
data_mean.mean(axis=1, level='subject')

subject,Bob,Guido,Sue
year,,,
2013,33.075,35.625,37.575
2014,37.900,36.000,36.750


In [10]:
def make_df(cols, ind):
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

In [11]:
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [12]:
df1 = make_df('AB', [0, 1])
df2 = make_df('AB', [2, 4])
print(df1); print(df2); print(pd.concat([df1, df2], axis=1))

    A   B
0  A0  B0
1  A1  B1
    A   B
2  A2  B2
4  A4  B4
     A    B    A    B
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
2  NaN  NaN   A2   B2
4  NaN  NaN   A4   B4
